# ACE Exploration

ACE (Advanced Composition Explorer) is equipped with nine scientific instruments to make comprehensive and coordinated in situ measurements. These instruments are categorized into two groups: High Resolution Spectrometers and Monitoring Instruments.

## High Resolution Spectrometers
- **CRIS** - Cosmic Ray Isotope Spectrometer
- **SIS** - Solar Isotope Spectrometer
- **ULEIS** - Ultra Low Energy Isotope Spectrometer
- **SEPICA** - Solar Energetic Particle Ionic Charge Analyzer
- **SWICS** - Solar Wind Ion Composition Spectrometer
- **SWIMS** - Solar Wind Ion Mass Spectrometer

## Monitoring Instruments
- **MAG** - Magnetic Field Monitor
- **SWEPAM** - Solar Wind Electron, Proton and Alpha Monitor
- **EPAM** - Electron, Proton and Alpha Monitor
- **SWICS** - Solar Wind Ion Composition Spectrometer

All open-source ACE data are formatted using hierarchical data format (HDF). The data are organized by instrument and by time-averaging periods. Each instrument's data are stored in separate HDF data files, and separate HDF files also contain the data from the different averaging periods. For most of the instruments, the data are averaged hourly, daily, and per 27 days (1 Bartels rotation).

## About Hierarchical Data Formats
Hierarchical Data Formats (HDF) are open source file formats that support large, complex, heterogeneous data. HDF files use a “file directory” like structure that allows you to organize data within the file in many different structured ways, as you might do with files on your computer. HDF files also allow for embedding of metadata making them self-describing.

---

## Analytical Questions
How can we apply novel dimension reduction methods, such as PCA, TSNE, etc., to obtain informative solar wind in-situ data representation in low-dimensional space? How can this low-dimensional representation provide better 2D/3D visualization support than traditional dimension reduction techniques?

## Libraries and global variables

In [ ]:
# Standard library imports
import sys
import os

# Third-party imports
from contextlib import suppress
import warnings
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.decomposition import PCA, KernelPCA
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
# from keras.layers import Input, Dense
# from keras.models import Model

# Local application imports
sys.path.append("../src/scripts")
from utilities import (
    parse_hdf_data,
    merge_dataframes,
    missing_occurrences,
    sort_columns_except_key,
    visualize_flag,
    add_datetime_column,
)

# Set the warning filter to ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
# global variables
MISSING_FLAG = -999.900
N_SPLITS = 4

## Data Import

In [ ]:
# read data
data_dir = "../data/ace/raw"
swics_1hr_dir = f"{data_dir}/swics_1hr"
swics_2hr_dir = f"{data_dir}/swics_2hr"

mag_df = parse_hdf_data(f"{data_dir}/MAG_data_1hr.txt")
swepam_df = parse_hdf_data(f"{data_dir}/SWEPAM_data_1hr.txt")
epam_df = parse_hdf_data(f"{data_dir}/EPAM_data_1hr.txt")

swics_dfs = []
for dir in [swics_1hr_dir, swics_2hr_dir]:
    for file in os.listdir(dir):
        swics_dfs.append(parse_hdf_data(f"{dir}/{file}"))
swics_df = pd.concat(swics_dfs)

In [ ]:
ACE_DATASETS = [mag_df, swepam_df, epam_df, swics_df]
ACE_DATASETS_NAMES = ["MAG", "SWEPAM", "EPAM", "SWICS"]

In [ ]:
# dtype conversion
for df in ACE_DATASETS:
    df[["year", "day", "hr", "min", "sec"]] = df[
        ["year", "day", "hr", "min", "sec"]
    ].astype(int)

    with suppress(KeyError):
        df['Quality'] = df['Quality'].astype(str)

In [ ]:
# datetime conversion and drop redundant features
for df in ACE_DATASETS:
    add_datetime_column(df).drop(
        columns=["year", "day", "hr", "min", "sec", "fp_year", "fp_doy"],
        inplace=True,
        axis=1,
    )

# swics_df may contain duplicate records to nature of 1.0 and 2.0 data collection
swics_df.drop_duplicates(subset="datetime", inplace=True)

## Data Cleaning

### Descriptives

In [ ]:
for df, df_name in zip(ACE_DATASETS, ACE_DATASETS_NAMES):
    print(f"Dataframe: {df_name}")
    display(df.info())
    display(df.describe())
    print("\n" + ("-" * 20))

### Retain *Good* Quality data

Good data is flagged by the researchers with a value of 0. 

In [ ]:
for c, (df, df_name) in enumerate(zip(ACE_DATASETS, ACE_DATASETS_NAMES)):

    with suppress(KeyError):  # not all datasets have the quality flag
        if df_name != "SWICS":
            df = df[df["Quality"] == "0.0"]
            df.drop(columns=["Quality"], inplace=True, axis=1)
        else:
            qf_cols = swics_df.filter(regex="^qf_").columns
            df = swics_df[
                (swics_df[qf_cols].isna() | swics_df[qf_cols].eq(0)).any(axis=1)
            ]

    ACE_DATASETS[c] = df

mag_df, swepam_df, epam_df, swics_df = ACE_DATASETS

### Join data

In [ ]:
# find unique timestamps
mag_dates, swepam_dates, epam_dates, swics_dates = [
    df.datetime.unique() for df in ACE_DATASETS
]

# find the common dates for 1hr interval data
common_dates_1hr = reduce(
    np.intersect1d, (mag_dates, swepam_dates, epam_dates)
)

# find the common dates for 2hr interval data
common_date_2hr = reduce(
    np.intersect1d, (mag_dates, swepam_dates, epam_dates, swics_dates)
    )

print(len(common_dates_1hr))
print(len(common_date_2hr))

In [ ]:
# join the 1hr to 2 hr interval datasets
insitu_df = merge_dataframes(ACE_DATASETS, "datetime")
df = sort_columns_except_key(insitu_df, "datetime")


### Handling Missing Values

Missing data has the value of -999.900. Assert that there are no longer missing values due to dropping data labeled as not of good quality.

In [ ]:
df_missing = missing_occurrences(df, MISSING_FLAG).sort_values(
    ascending=False, by="Missing_Count"
)
visualize_flag(df_missing)
df.shape

#### Imputation Methods 

Univariate time series imputation methods: 
- Mean (median)
- Last Observed Carried Forward
- Linear Interpolation
- Polynomial Interpolation
- Kalman Filter
- Moving Averge
- Random

Multivariate Time Series Imputation methods. 
- K-Nearest Neighbords
- Random Forest
- Multiple Singular Spectral Analysis
- Expectation-Maximization
- Multiple Imputation with Chained Equations

In [ ]:
# Prepare for imputation
numeric_df = df.select_dtypes(include=[np.number])
df = df.replace(-9999.9, np.nan)
X_incomplete = df.select_dtypes(include=[np.number])

# KNN Imputer with 3 neighbors
knn_imputer = KNNImputer(n_neighbors=3, add_indicator=True)
knn_imputed_data = knn_imputer.fit_transform(X_incomplete)
df_knn_imputed = pd.DataFrame(knn_imputed_data, columns=numeric_df.columns)
df_knn_imputed.insert(0, "datetime", df["datetime"])

# Create the Random Forest imputer
rf_imputer = IterativeImputer(
    estimator=RandomForestRegressor(), random_state=0, max_iter=10
)
rf_imputed_data = rf_imputer.fit_transform(X_incomplete)
df_knn_imputed = pd.DataFrame(rf_imputed_data, columns=numeric_df.columns)
df_knn_imputed.insert(0, "datetime", df["datetime"])

In [ ]:
# Create the Random Forest imputer
rf_imputer = IterativeImputer(
    estimator=RandomForestRegressor(), random_state=0, max_iter=10
)

# Perform the imputation
imputed_data = rf_imputer.fit_transform(X_incomplete)

## Data Transformation

### Log transformation of all quantities

In [ ]:
# TODO: select the subset that needs log transformation
df.apply(np.log10)

## Dimensionality Reduction

### Dimensionality Reduction Using PCA

In [ ]:
# Fit PCA
pca = PCA().fit(X)

# Plot the explained variances
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color="black")
plt.xlabel("PCA features")
plt.ylabel("variance %")
plt.xticks(features)

# Save components to a DataFrame
PCA_components = pd.DataFrame(pca.transform(X))

plt.show()

In [ ]:
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
num_components = np.where(cumulative_variance > 0.95)[0][0] + 1
print("Number of components to explain 95% Variance: ", num_components)

In [ ]:
# Create a PCA that will retain 2 components
pca = PCA(n_components=num_components, whiten=True)

# Conduct PCA
X_pca = pca.fit_transform(X)

# Show the new data
print("original shape:   ", X.shape)
print("transformed shape:", X_pca.shape)

# The transformed data has been reduced to two dimensions
df = pd.DataFrame(
    data=X_pca,
)
print(df.head())

### Dimensionality Reduction Using Kernel PCA

In [ ]:
# Fit Kernel PCA with n_components=None to compute all components
kpca = KernelPCA(n_components=None, kernel="rbf")
kpca.fit(X)

# Get eigenvalues
eigenvalues = kpca.lambdas_

# Plot eigenvalues
plt.plot(eigenvalues, "bo-")
plt.xlabel("Index")
plt.ylabel("Eigenvalue")
plt.show()

In this plot, the x-axis represents the index of each component (in descending order of eigenvalue), and the y-axis represents the corresponding eigenvalue. You typically choose the number of components at the point where adding another component doesn't significantly increase the eigenvalue (the "elbow" of the plot).

### Dimensionality Reduction Using Autoencoders

In [ ]:
# Define the size of the encoded representation
encoding_dim = 2  # 2-dimensional encoded representation

# Define the input layer
input_img = Input(shape=(X.shape[1],))

# Define the encoded layer
encoded = Dense(encoding_dim, activation="relu")(input_img)

# Define the decoded layer
decoded = Dense(X.shape[1], activation="sigmoid")(encoded)

# Define the autoencoder model
autoencoder = Model(input_img, decoded)

# Define the encoder model
encoder = Model(input_img, encoded)

# Define the decoder model
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

# Compile the autoencoder
autoencoder.compile(optimizer="adadelta", loss="binary_crossentropy")

# Train the autoencoder
autoencoder.fit(X, X, epochs=50, batch_size=256, shuffle=True)

# Use the encoder to reduce the dimensionality of the data
X_encoded = encoder.predict(X)

print("original shape:   ", X.shape)
print("transformed shape:", X_encoded.shape)

## Self-Organizing Maps